In [26]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn.metrics import confusion_matrix

import utils.text_processing as tp

In [12]:
pmethod = 'typos'

In [13]:
%store -r important_words
%store -r sentence_packages
%store -r ds
%store -r advds
%store -r extended_original_predictions
%store -r extended_modified_predictions

In [42]:
sentence_packages = [{'original_sentence': 'This computer is absolutely AMAZING!!!', 'modified_sentences': [('This', 'computer is absolutely AMAZING!!!'), ('computer', 'This is absolutely AMAZING!!!'), ('is', 'This computer absolutely AMAZING!!!'), ('absolutely', 'This computer is AMAZING!!!'), ('AMAZING!!!', 'This computer is absolutely')]}, {'original_sentence': '10 plus hours of battery...', 'modified_sentences': [('10', 'plus hours of battery...'), ('plus', '10 hours of battery...'), ('hours', '10 plus of battery...'), ('of', '10 plus hours battery...'), ('battery...', '10 plus hours of')]}]

In [43]:
len(sentence_packages)

2

In [ ]:
if modified_words[index] is None:
            modified_sentences.append(sentence)
            continue  
               
        
            
        if isinstance(modified_words[index], list): 
            modified_sentences_list = []
            for word in modified_words[index]:
                modified_sentences_list.append(sentence.replace(important_words[index], word))
            modified_sentences.append(modified_sentences_list)               
            continue    
        print(important_words[index], modified_words[index])
        modified_sentences.append(sentence.replace(important_words[index], modified_words[index]))  

In [198]:
original_sentences = ['This computer is absolutely AMAZING!!!',
                         '10 plus hours of battery...',
                         'super fast processor and really nice graphics card..',
                         'super fast processor and really nice graphics card..',
                         'and plenty of storage with 250 gb(though I will upgrade this and the ram..)']
important_words_list = [[], [], ['card..'], ['card..'], ['plenty', 'storage']]
modified_words_list = [[], [], [['c4rd..', 'COOOD']], [['c4rd..']], [['p13nty'], ['5t0r4g3']]]

In [199]:
def generate_modified_sentences(original_sentences, important_words_list, modified_words_list):
    
    modified_sentences_list = []
    for sent_idx, sentence in enumerate(original_sentences):
        # print(sent_idx, sentence)
        # print(important_words_list[sent_idx])
                
        modified_sentences = []
        for iword_idx, iword in enumerate(important_words_list[sent_idx]):
            for mwords in modified_words_list[sent_idx]:
                for mword in mwords:               
                      
                    modified_sentences.append(sentence.replace(iword, mword))
        modified_sentences_list.append(modified_sentences)
        
    return modified_sentences_list

In [200]:
modified_sentences_list = generate_modified_sentences(original_sentences, important_words_list, modified_words_list)

In [201]:
modified_sentences_list

[[],
 [],
 ['super fast processor and really nice graphics c4rd..',
  'super fast processor and really nice graphics COOOD'],
 ['super fast processor and really nice graphics c4rd..'],
 ['and p13nty of storage with 250 gb(though I will upgrade this and the ram..)',
  'and 5t0r4g3 of storage with 250 gb(though I will upgrade this and the ram..)',
  'and plenty of p13nty with 250 gb(though I will upgrade this and the ram..)',
  'and plenty of 5t0r4g3 with 250 gb(though I will upgrade this and the ram..)']]